In [ ]:
# 获取当前notebook的绝对路径
import sys, os
# import asyncio
# import nest_asyncio
# nest_asyncio.apply()
from pathlib import Path
current_path = Path.cwd()  # 或者使用 %pwd magic命令
project_root = current_path.parents[2]  # 如果notebook在project_root/notebooks/下
print("notebook所在的路径：", current_path)
print("project_root：", project_root)
sys.path.append(str(project_root))
import FastAPI_setup

In [3]:
# project_id = "20563b61-7818-474b-9641-c40757bbd3fe"  # test1
project_id = "cc6bca8a-df5d-43d5-a931-875113df1f50"  # test2

In [ ]:
# 使用Celery任务在后台启动分析流程
from app.tasks.structuring_tasks import run_structuring_analysis
celery_task = run_structuring_analysis.delay(project_id)

注意： celery的任务是非阻塞执行，所以没有等到任务执行完，就会先返回一个AsyncResult对象。这个对象里，只有一个UUID作为任务标识符。

In [ ]:
print(celery_task)

想要看真正的结果，得等执行完以后，使用get()方法获取结果。
result = celery_task.get()

In [ ]:
result = celery_task.get()
print(result)